In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
from tensorflow.keras import layers

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
# Load a dataset into a Pandas Dataframe
dataset_df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
print("Full train dataset shape is {}".format(dataset_df.shape))

dataset_df = dataset_df.drop(['PassengerId', 'Name'], axis=1)
dataset_df[["Deck", "Cabin_num", "Side"]] = dataset_df["Cabin"].str.split("/", expand=True)
dataset_df = dataset_df.drop('Cabin', axis=1)
dataset_df.fillna(0, inplace=True)
dataset_df['Transported'] = dataset_df['Transported'].astype(int)
dataset_df['VIP'] = dataset_df['VIP'].astype(int)
dataset_df['CryoSleep'] = dataset_df['CryoSleep'].astype(int)
dataset_df.head(5)
dataset_df.isna().sum().sort_values(ascending=False)

Full train dataset shape is (8693, 14)


HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
Deck            0
Cabin_num       0
Side            0
dtype: int64

In [4]:
def split_dataset(dataset, test_ratio=0.20):
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

train_ds_pd, valid_ds_pd = split_dataset(dataset_df)
print("{} examples in training, {} examples in testing.".format(
    len(train_ds_pd), len(valid_ds_pd)))

6941 examples in training, 1752 examples in testing.


In [5]:
def convert_columns(dataframe, target=None, features=None):
    for name in dataframe.columns.values:
        if name != target:
            if is_numeric_dtype(dataframe[name]):
                dataframe[name] = dataframe[name].astype('float')
                if features != None:
                    feature_columns.append(tf.feature_column.numeric_column(name))
            else:
                dataframe[name] = dataframe[name].astype('string')
                dataframe[name] = dataframe[name].astype('category')
                if features != None:
                    features.append(tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_vocabulary_list(name, vocabulary_list=dataframe[name].unique()), dimension=8))

In [6]:
feature_columns = []
convert_columns(train_ds_pd, 'Transported', feature_columns)
convert_columns(valid_ds_pd, 'Transported')

/tmp/ipykernel_20/3352524088.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[name] = dataframe[name].astype('string')
/tmp/ipykernel_20/3352524088.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[name] = dataframe[name].astype('category')
/tmp/ipykernel_20/3352524088.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

In [7]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, target=None, shuffle=True, batch_size=32):
    df = dataframe.copy()
    if target != None:
        labels = df.pop(target)
    #df = {key: value for key, value in dataframe.items()}
    
    if target != None:
        ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    else:
        ds = tf.data.Dataset.from_tensor_slices(dict(df))
        
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    
    return ds

In [8]:
train_ds_nn = df_to_dataset(train_ds_pd, 'Transported')
valid_ds_nn = df_to_dataset(valid_ds_pd, 'Transported', shuffle=False)

In [9]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
  normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Create a layer that turns strings into integer indices.
  if dtype == 'string':
    index = layers.StringLookup(max_tokens=max_tokens)
  # Otherwise, create a layer that turns integer values into integer indices.
  else:
    index = layers.IntegerLookup(max_tokens=max_tokens)

  # Prepare a `tf.data.Dataset` that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the set of possible values and assign them a fixed integer index.
  index.adapt(feature_ds)

  # Encode the integer indices.
  encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())

  # Apply multi-hot encoding to the indices. The lambda function captures the
  # layer, so you can use them, or include them in the Keras Functional model later.
  return lambda feature: encoder(index(feature))

In [10]:
cat_types = ['HomePlanet', 'Destination', 'Deck', 'Cabin_num', 'Side']
num_types = [lab for lab in train_ds_pd.columns.values if lab not in cat_types and lab != 'Transported']
all_inputs = []
encoded_features = []

In [11]:
for header in num_types:
    numeric_col = tf.keras.Input(shape=(1,), name=header)
    normalization_layer = get_normalization_layer(header, train_ds_nn)
    encoded_numeric_col = normalization_layer(numeric_col)
    all_inputs.append(numeric_col)
    encoded_features.append(encoded_numeric_col)

In [12]:
for header in cat_types:
    categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
    encoding_layer = get_category_encoding_layer(name=header,
                                               dataset=train_ds_nn,
                                               dtype='string',
                                               max_tokens=5)
    encoded_categorical_col = encoding_layer(categorical_col)
    all_inputs.append(categorical_col)
    encoded_features.append(encoded_categorical_col)

In [13]:
nn_model = tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(feature_columns),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='softmax')
])

nn_model.compile(optimizer='Adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

nn_model.fit(train_ds_nn,
             validation_data=valid_ds_nn,
             epochs=20)

Epoch 1/20
217/217 [==============================] - 5s 8ms/step - loss: 46.1881 - accuracy: 0.5009 - val_loss: 7.4071 - val_accuracy: 0.5143
Epoch 2/20
217/217 [==============================] - 1s 4ms/step - loss: 11.0187 - accuracy: 0.5009 - val_loss: 6.0296 - val_accuracy: 0.5143
Epoch 3/20
217/217 [==============================] - 1s 4ms/step - loss: 8.3901 - accuracy: 0.5009 - val_loss: 4.3367 - val_accuracy: 0.5143
Epoch 4/20
217/217 [==============================] - 1s 4ms/step - loss: 6.3586 - accuracy: 0.5009 - val_loss: 3.2804 - val_accuracy: 0.5143
Epoch 5/20
217/217 [==============================] - 1s 4ms/step - loss: 4.2150 - accuracy: 0.5009 - val_loss: 2.5716 - val_accuracy: 0.5143
Epoch 6/20
217/217 [==============================] - 1s 4ms/step - loss: 3.2095 - accuracy: 0.5009 - val_loss: 1.6970 - val_accuracy: 0.5143
Epoch 7/20
217/217 [==============================] - 1s 4ms/step - loss: 2.0549 - accuracy: 0.5009 - val_loss: 1.3584 - val_accuracy: 0.5143
Epoc

### Predict

In [14]:
df_test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
df_test = df_test.drop(['PassengerId', 'Name'], axis=1)
df_test[["Deck", "Cabin_num", "Side"]] = df_test["Cabin"].str.split("/", expand=True)
df_test = df_test.drop('Cabin', axis=1)
df_test.fillna(0, inplace=True)
df_test['VIP'] = df_test['VIP'].astype(int)
df_test['CryoSleep'] = df_test['CryoSleep'].astype(int)
df_test.head(5)
df_test.isna().sum().sort_values(ascending=False)

#for name in df_test.columns.values:
#    if ~is_numeric_dtype(df_test[name]):
#        df_test[name] = df_test[name].astype('category').cat.codes

convert_columns(df_test)
test_ds_nn = df_to_dataset(df_test, shuffle=False)
#df = {key: value for key, value in df_test.items()}
#ds = tf.data.Dataset.from_tensor_slices(dict(df))
#test_ds_nn = tf.data.Dataset.from_tensor_slices(dict(df_test)).batch(32)

In [15]:
preds = nn_model.predict(test_ds_nn)

134/134 [==============================] - 1s 3ms/step


In [16]:
predictions = [0]*len(preds)
predictions = [1 for pred in preds if pred > 0.5]
predictions

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
